In [1]:
import os
import pandas as pd
import datetime


path = r"E:\ML algoritme tl anfaldsdetektion vha HRV\LabView-Results\Patient5_1-corrected-rr.lvm"



In [2]:
# Load RR intervals from file (header lines 1-20, data starts at line 23)
# The file uses tabs as separators and comma as decimal separator (e.g. "0,000000\t0,244154")
skiprows = 22

# read with explicit tab separator and decimal comma; fallback to whitespace if needed
try:
    df_raw = pd.read_csv(path, skiprows=skiprows, sep='\t', header=0, engine='python', decimal=',')
except Exception:
    df_raw = pd.read_csv(path, skiprows=skiprows, header=0, delim_whitespace=True, engine='python', decimal=',')

# If no header row was present, ensure columns are strings and try again using first row as header
if df_raw.columns.to_series().astype(str).str.contains('[A-Za-z]').any() is False and df_raw.iloc[0].astype(str).str.contains('[A-Za-z]').any():
    df_raw = pd.read_csv(path, skiprows=skiprows, header=0, sep='\t', engine='python', decimal=',')

# Prefer numeric-typed columns; otherwise coerce columns to numeric (handling comma decimals)
num_cols = df_raw.select_dtypes(include=['number']).columns
if len(num_cols) > 0:
    rr = df_raw[num_cols[1]].dropna().reset_index(drop=True)
else:
    # coerce by replacing comma with dot then converting
    coerced = df_raw.apply(lambda col: pd.to_numeric(col.astype(str).str.replace(',', '.'), errors='coerce'))
    numeric_mask = coerced.notna().sum() > 0
    if numeric_mask.any():
        rr = coerced.loc[:, numeric_mask].iloc[:, 0].dropna().reset_index(drop=True)
    else:
        # fallback: first column
        rr = pd.to_numeric(df_raw.iloc[:, 0].astype(str).str.replace(',', '.'), errors='coerce').dropna().reset_index(drop=True)

rr.name = 'RR'
print(f"Loaded {len(rr)} RR intervals")
rr.head()

Loaded 186084 RR intervals


0    0.331858
1    0.382904
2    0.167871
3    0.204339
4    0.220332
Name: RR, dtype: float64

In [3]:
# Merge RR intervals shorter than 0.27 s into the smaller adjacent interval.
# Uses existing `rr` Series from the notebook.
threshold = 60/220

vals = rr.tolist()
removed = 0

while True:
    # find first short interval
    short_idx = next((i for i, v in enumerate(vals) if v < threshold), None)
    if short_idx is None:
        break

    v = vals[short_idx]
    # choose neighbor (previous or next) with the smaller value (handle edges)
    if short_idx == 0:
        neighbor = 1
    elif short_idx == len(vals) - 1:
        neighbor = short_idx - 1
    else:
        neighbor = short_idx - 1 if vals[short_idx - 1] <= vals[short_idx + 1] else short_idx + 1

    # add short interval to chosen neighbor and remove it
    vals[neighbor] += v
    vals.pop(short_idx)
    removed += 1

rr_clean = pd.Series(vals, name='RR')
print(f"Removed {removed} short intervals. Before: {len(rr)}, After: {len(rr_clean)}")

Removed 13435 short intervals. Before: 186084, After: 172649


In [4]:
print(len(rr_clean))

172649


In [5]:
rr_clean_total = 0

for i in rr_clean:
    rr_clean_total += i

print(f"Total duration of cleaned RR intervals: {rr_clean_total} seconds")

Total duration of cleaned RR intervals: 160685.02818899963 seconds


In [6]:
fs = 512
s_total = 82460288/fs
print(f"Total duration based on original signal sampling rate: {s_total} seconds")

remove_start = 60
print(f"Samples to remove at start: {remove_start}")
remove_end = 300
print(f"Samples to remove at end: {remove_end}")

s_effective = s_total - (remove_start + remove_end)
print(f"Effective duration after removing start/end: {s_effective} seconds")

Total duration based on original signal sampling rate: 161055.25 seconds
Samples to remove at start: 60
Samples to remove at end: 300
Effective duration after removing start/end: 160695.25 seconds


In [11]:
import neurokit2 as nk
import nptdms

# Read TDMS file
try:
    tdms = nptdms.TdmsFile.read(path_patient)
except Exception as e:
    raise RuntimeError(f"Failed to read TDMS file: {e}")

# List groups
groups = tdms.groups()
print("Groups found:", [g.name for g in groups])

# Collect channel data into a dict of pandas Series
channels = {}
tdms_time_printed = globals().get('tdms_time_printed', False)

for g in groups:
    for ch in g.channels():
        key = f"{g.name}.{ch.name}"
        try:
            arr = ch[:]  # numpy array of channel samples
        except Exception:
            arr = ch.data  # fallback
        channels[key] = pd.Series(arr, name=key)

        # Print "Time" from TDMS header/properties once
        if not tdms_time_printed:
            time_val = None

            # file-level properties
            props = getattr(tdms, "properties", {}) or {}
            if isinstance(props, dict):
                for k, v in props.items():
                    if "time" in str(k).lower():
                        time_val = v
                        break

            # fallback: file_info dict-like
            if time_val is None and hasattr(tdms, "file_info"):
                fi = getattr(tdms, "file_info") or {}
                if isinstance(fi, dict):
                    for k, v in fi.items():
                        if "time" in str(k).lower():
                            time_val = v
                            break

            # channel-level properties
            if time_val is None and hasattr(ch, "properties"):
                ch_props = getattr(ch, "properties") or {}
                if isinstance(ch_props, dict):
                    for k, v in ch_props.items():
                        if "time" in str(k).lower():
                            time_val = v
                            break

            if time_val is not None:
                print("Time from header:", time_val)
            else:
                print("No 'Time' property found in TDMS header/channel properties")

            globals()["tdms_time_printed"] = True
            tdms_time_printed = True

# Build DataFrame (will align by index; if lengths differ, shorter series get NaN)
df_tdms = pd.concat(channels, axis=1)

print(f"Loaded TDMS with shape: {df_tdms.shape}")
df_tdms.head()


Groups found: ['Untitled']
Time from header: 2016-10-12T09:05:02.000000
Loaded TDMS with shape: (82460288, 1)


,Untitled.EKG
0,-70.270817
1,-1.377859
2,37.202197
3,-45.469352
4,-144.675212


In [8]:
# Clean ECG signal (uses existing `df_tdms` and `key` variables)
signal = df_tdms[key]  # 'Untitled.EKG'
ecg_cleaned = nk.ecg_clean(signal, method="neurokit")  # adjust method/sampling_rate if needed

# Save cleaned signal to dataframe and show a quick check
df_tdms["EKG_clean"] = ecg_cleaned
print(f"Cleaned ECG signal: {len(ecg_cleaned)} samples. Added column 'EKG_clean' to df_tdms.")
df_tdms[["Untitled.EKG", "EKG_clean"]].head()

Cleaned ECG signal: 82460288 samples. Added column 'EKG_clean' to df_tdms.


,Untitled.EKG,EKG_clean
0,-70.270817,-549.314036
1,-1.377859,-553.209804
2,37.202197,-557.270345
3,-45.469352,-561.564011
4,-144.675212,-565.635597


In [9]:
import numpy as np

# Try to infer sampling rate from TDMS channel properties (common property names)
srate = None
if 'ch' in globals() and hasattr(ch, "properties"):
    for key in ("wf_sample_rate", "sample_rate", "sampling_rate", "Rate", "rate"):
        if key in ch.properties and ch.properties[key] not in (None, ""):
            srate = ch.properties[key]
            break

# Normalize srate type if present
try:
    if isinstance(srate, (bytes, bytearray)):
        srate = float(srate.decode())
    elif srate is not None:
        srate = float(srate)
except Exception:
    srate = None

# Fallback sampling rate (adjust if you know the true rate)
if srate is None:
    srate = 512.0  # change this to the correct sampling rate if known
print(f"Using sampling rate = {srate} Hz")

# Detect R-peaks with NeuroKit2
signals, info = nk.ecg_peaks(ecg_cleaned, sampling_rate=int(srate))

# Extract R-peak indices (NeuroKit returns a dict under key 'ECG_R_Peaks')
if isinstance(info, dict) and "ECG_R_Peaks" in info:
    rpeak_indices = np.asarray(info["ECG_R_Peaks"])
else:
    # fallback: try to extract first array found in `info`
    if isinstance(info, dict) and len(info) > 0:
        first_val = next(iter(info.values()))
        rpeak_indices = np.asarray(first_val)
    else:
        # if something unexpected, make empty array
        rpeak_indices = np.array([], dtype=int)

print(f"Detected {len(rpeak_indices)} R-peaks")

# Compute times of R-peaks (seconds)
times = np.arange(len(ecg_cleaned)) / float(srate)
rpeak_times = times[rpeak_indices]

# Optionally mark R-peaks in df_tdms if lengths match
if 'df_tdms' in globals() and len(df_tdms) == len(ecg_cleaned):
    df_tdms["R_Peak"] = False
    df_tdms.loc[rpeak_indices, "R_Peak"] = True
    print("Added boolean 'R_Peak' column to df_tdms.")
else:
    if 'df_tdms' in globals():
        print("df_tdms length does not match signal length; not adding 'R_Peak' column.")

# Expose results in variables for further use
rpeaks_indices = rpeak_indices
rpeaks_times = rpeak_times

# Quick preview
print("First 10 R-peak indices:", rpeaks_indices[:10])
print("First 10 R-peak times (s):", rpeaks_times[:10])

Using sampling rate = 512.0 Hz
Detected 167333 R-peaks
Added boolean 'R_Peak' column to df_tdms.
First 10 R-peak indices: [1218 2387 2785 3037 4280 4991 6417 6817 8551 9145]
First 10 R-peak times (s): [ 2.37890625  4.66210938  5.43945312  5.93164062  8.359375    9.74804688
 12.53320312 13.31445312 16.70117188 17.86132812]
